In [39]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException,NoSuchElementException, ElementNotInteractableException
import time
from bs4 import BeautifulSoup
import csv
from selenium.webdriver.chrome.service import Service
import os
import json
import random


# 读取 diff.csv 文件中的 MMSI 列表
mmsi_list = pd.read_csv('diff.csv')['mmsi'].tolist()
current_index=0
csv_num=1
index_in_current_csv=0

with open('ship_data'+str(csv_num)+'.csv', 'a', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(['IMO','MMSI', 'Callsign', 'Vessel Name', 'Ship type', 'Flag',  'Length Overall (m)', 'Beam (m)'])


def save_progress(current_index,csv_num,index_in_current_csv):
    progress_data={
        'current_index':current_index,
        'csv_num':csv_num,
        'index_in_current_csv':index_in_current_csv
    }
    with open('progress.json','w') as f:
        json.dump(progress_data,f,indent=4)

def load_progress():
    if os.path.exists('progress.json'):
        global current_index,csv_num,index_in_current_csv
        with open('progress.json','r')as f:
            progress_data=json.load(f)
            current_index=progress_data['current_index']
            csv_num=progress_data['csv_num']
            index_in_current_csv=progress_data['index_in_current_csv']


# 设置 ChromeDriver 的路径
chrome_driver_path = 'C:/driver/chromedriver-win64/chromedriver.exe'  # 替换为 ChromeDriver 的实际路径

# 初始化 Selenium WebDriver
service = Service(executable_path=chrome_driver_path)
driver = webdriver.Chrome(service=service)

driver.get("http://www.ships66.com/#/")

# 等待网站加载
time.sleep(5)
# 存储结果数据


In [40]:
# 定义函数，输入mmsi并搜索
def serach_left_table(mmsi):
    try:
        search_box = driver.find_element(By.ID, 'searchIn')
        search_box.clear()
        search_box.send_keys(mmsi)

        # 点击搜索按钮
        search_button = driver.find_element(By.CLASS_NAME, 'searchbtn')
        search_button.click()

        try:
            # 等待搜索结果加载并显示
            WebDriverWait(driver, 4).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'bottomitem'))
            )
        except TimeoutException:
            print(f"can find any ship for MMSI: {mmsi} ")
            return False

        # 获取并点击搜索结果
        search_result_xpath = "//div[@class='all_search ac_results']//li/a"
        search_result = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, search_result_xpath))
        )
        
        # 滚动到元素并确保其可见
        driver.execute_script("arguments[0].scrollIntoView(true);", search_result)
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, search_result_xpath)))
        
        # 重新获取元素引用以避免 StaleElementReferenceException
        search_result = driver.find_element(By.XPATH, search_result_xpath)
        search_result.click()
        return True
    except (TimeoutException, NoSuchElementException, ElementNotInteractableException, StaleElementReferenceException) as e:
        print(f"Error: {str(e)}")
        return False




In [41]:
def serach_right_table(mmsi):
    # 等待详细信息弹出窗口加载
    global index_in_current_csv
    try:
        # 使用更具体的 XPath 选择器
        element = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.XPATH, "//div[contains(@class, 'ships66-layer') and contains(@class, 'right-top-pop')]"))
        )
        print(f"Element found for MMSI: {mmsi}, element: {element}")

        # # 使用 JavaScript 滚动到元素位置
        # driver.execute_script("arguments[0].scrollIntoView(true);", element)

    except TimeoutException:
        print(f"TimeoutException occurred for MMSI: {mmsi} while waiting for detail popup")
        with open(f'html_now_{mmsi}.html', 'w', encoding='utf-8') as record:
            record.write(driver.page_source)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 解析详细信息弹出窗口中的数据
    detail_layer = soup.find('div', class_='ships66-layer')
    if detail_layer:
        imo_info = mmsi_info = Callsign = flag = name = ship_type = length = width = '-'
        name = detail_layer.find('span', {'title': True}).get('title', '').strip()
        mmsi_info=mmsi
        for item in detail_layer.find_all('li', class_='flex-in flex-c'):
            title = item.find('span', class_='title').get('title', '').strip()
            value = item.find_all('span')[-1].get('title', '').strip()
            if title == 'IMO':
                imo_info = value
            elif title == '呼号':
                Callsign = value
            elif title == '船型':
                ship_type = value
            elif title == '船籍':
                flag = value
            elif title == '长度':
                length = value
            elif title == '宽度':
                width = value
        this_ship=[imo_info,mmsi_info,Callsign,name,ship_type,flag,length,width]
        print('current ship data:',this_ship)
        index_in_current_csv+=1
        
    return this_ship



In [44]:
load_progress()

for current_index in range(current_index,len(mmsi_list)):
    mmsi=mmsi_list[current_index]
    print(mmsi)
    results = []
    if index_in_current_csv>=4300:
        csv_num+=1
        with open('ship_data'+str(csv_num)+'.csv', 'a', newline='', encoding='utf-8-sig') as file:
            writer = csv.writer(file)
            writer.writerow(['IMO','MMSI', 'Callsign', 'Vessel Name', 'Ship type', 'Flag',  'Length Overall (m)', 'Beam (m)'])
        index_in_current_csv=0
    if serach_left_table(mmsi):
        with open('ship_data'+str(csv_num)+'.csv', 'a', newline='', encoding='utf-8-sig') as file:
            writer=csv.writer(file)
            writer.writerow(serach_right_table(mmsi))
        
    save_progress(current_index+1,csv_num,index_in_current_csv)
    time.sleep(random.uniform(1,3))
# 关闭浏览器
driver.quit()


244710523


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF7A8E8EEB2+31554]
	(No symbol) [0x00007FF7A8E07EE9]
	(No symbol) [0x00007FF7A8CC872A]
	(No symbol) [0x00007FF7A8C9D995]
	(No symbol) [0x00007FF7A8D444D7]
	(No symbol) [0x00007FF7A8D5C051]
	(No symbol) [0x00007FF7A8D3CDD3]
	(No symbol) [0x00007FF7A8D0A33B]
	(No symbol) [0x00007FF7A8D0AED1]
	GetHandleVerifier [0x00007FF7A9198B2D+3217341]
	GetHandleVerifier [0x00007FF7A91E5AF3+3532675]
	GetHandleVerifier [0x00007FF7A91DB0F0+3489152]
	GetHandleVerifier [0x00007FF7A8F3E786+750614]
	(No symbol) [0x00007FF7A8E1376F]
	(No symbol) [0x00007FF7A8E0EB24]
	(No symbol) [0x00007FF7A8E0ECB2]
	(No symbol) [0x00007FF7A8DFE17F]
	BaseThreadInitThunk [0x00007FF8E622257D+29]
	RtlUserThreadStart [0x00007FF8E6F4AF28+40]
